<a href="https://colab.research.google.com/github/Magaton1010/Python_Analysis/blob/main/Data_Segregation_23_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Segregation Tutorial

This repository contains a Python script that demonstrates the process of segregating data from multiple Excel files based on specific criteria. The script uses the pandas library for data manipulation and follows best practices for code readability and efficiency.

## Overview

The provided Python script reads Excel files from a specified input folder, performs data segregation based on a sample ID column, and creates new columns with transformed data. The script deletes unnecessary columns, creates new columns based on farm ID, and applies various transformations.

## Features

- Input folder and output folder paths are easily customizable.
- The script processes each Excel file in the input folder.
- Unnecessary columns are deleted to streamline the data.
- New columns are created based on the farm ID, including crop type, site, season, and more.
- The script handles different farm IDs with specific depth and treatment keys.
- The "Sampling Date" column is converted to a datetime object with a specific format.
- The final data is saved to new Excel files in the output folder.

## Instructions

1. Clone the repository to your local machine.
2. Install the required dependencies: `pip install pandas`.
3. Customize the input and output folder paths in the script.
4. Run the script to process your data.

## Usage Example

```python
# Example command to run the script
python data_segregation_script.py

In [ ]:
import pandas as pd
import glob
import os

# Set the folder paths for input and output files
input_folder = r''
output_folder = r''

# Get a list of all Excel files in the input folder
excel_files = glob.glob(os.path.join(input_folder, '*.xlsx'))

# Process each Excel file
for file in excel_files:
    # Read the Excel file
    data = pd.read_excel(file)

    # Ask for the columns to delete
    columns_to_delete = ['AcctNo', 'Company', 'FieldID', 'First', 'Last', 'Address', 'City', 'State', 'Zip', 'Country', 'Grower', 'FarmID', 'LabID', 'Received', 'Processed', 'Matrix']

    # Delete the specified columns
    data.drop(columns=columns_to_delete, inplace=True)

    # Ask for the sampleid column
    sampleid_column = 'SampleID'

    # Extract the data based on the fieldid column
    if sampleid_column in data.columns:
        # Get the first three characters of the fieldid column as the farm ID
        farm_id = data[sampleid_column].apply(lambda x: x.split('-')[0])

        # Create new columns based on the farm ID and perform the required transformations
        block_key = {'A': '1', 'B': '2', 'C': '3', 'D': '4'}
        crop_key = {'T1': 'TO', 'T2': 'TO', 'T3': 'TO', 'P1': 'PT', 'P2': 'PT', 'B1': 'GB', 'B2': 'GB'}
        site_key = {'T1': 'LL', 'T2': 'PI', 'T3': 'LE', 'P1': 'TI', 'P2': 'TL', 'B1': 'CC', 'B2': 'HW'}
        season_key = {'T1': 'F3', 'T2': 'W3', 'T3': 'W3', 'P1': 'W3', 'P2': 'W3', 'B1': 'W3', 'B2': 'W3'}
        spatial_key = {'DR': 'DR', 'FN': 'FN', 'SN': 'SN', 'ND': 'ND'}

        farm_ids = set(farm_id.tolist())
        if 'T1' in farm_ids or 'T2' in farm_ids or 'T3' in farm_ids:
            depth_key = {'S1': '15', 'S2': '30', 'S3': '45'}
            treatment_key = {'1': '0', '2': '50', '3': '75', '4': '100', '5': '150', '6': '200'}
        elif 'P1' in farm_ids or 'P2' in farm_ids:
            depth_key = {'S1': '20', 'S2': '40', 'S3': '60'}
            treatment_key = {'1': '0', '2': '46', '3': '92', '4': '137', '5': '183', '6': '229'}
        else:
            depth_key = {'S1': '5', 'S2': '30', 'S3': '45'}
            treatment_key = {'1': '0', '2': '40', '3': '80', '4': '120', '5': '113L'}

        # Create new columns based on the farm ID and perform the required transformations
        data["Crop"] = farm_id.map(crop_key)
        data["site"] = farm_id.map(site_key)
        data["season"] = farm_id.map(season_key)

        # Convert "Sampling Date" to a datetime object and format it as MM/DD/YYYY
        data["Sampling Date"] = pd.to_datetime(data[sampleid_column].apply(lambda x: x.split('-')[5]), format='%m%d%Y').dt.date

        data["sample_type"] = "S"

        # Extract the sample ID and create separate columns
        data["Block"] = data[sampleid_column].apply(lambda x: block_key.get(x.split('-')[1] if isinstance(x, str) else None))
        data["Treatment"] = data[sampleid_column].apply(lambda x: treatment_key.get(x.split('-')[2] if isinstance(x, str) else None))
        data["Row_type"] = data[sampleid_column].apply(lambda x: spatial_key.get(x.split('-')[3], None))
        data["Depth"] = data[sampleid_column].apply(lambda x: depth_key.get(x.split('-')[4], None))
        data["Time"] = data.apply(lambda row: row.get("Time", "N/A"), axis=1)

        # Create the new_sample_ID column
        data["new_sample_ID"] = data.apply(lambda row: f"{row['Crop']}-{row['site']}-{row['season']}-S{row.name + 1}", axis=1)

        # Sort the columns
        column_order = [
            sampleid_column,
            "new_sample_ID",
            "Sampling Date",
            "Crop",
            "site",
            "season",
            "Treatment",
            "Block",
            "Row_type",
            "Depth",
            "Time",
            "sample_type"
        ]

        # Create a list of columns that are not in column_order
        remaining_columns = [col for col in data.columns if col not in column_order]

        # Append the remaining columns to the column_order list
        column_order += remaining_columns

        # Reorder the columns in the DataFrame based on column_order
        data = data[column_order]

        # Print the modified data
        print("\nModified data:")
        print(data)

        # Create the output file path
        file_name = os.path.basename(file)
        output_file = os.path.join(output_folder, file_name.replace('.xlsx', '_Masterfile_preprocessed.xlsx'))

        # Save the modified data to a new Excel file
        data.to_excel(output_file, index=False)
        print(f"Processed data saved to '{output_file}'.")
    else:
        print("Fieldid column not found in the Excel file.")

In [ ]:
import pandas as pd
import re
import glob
import os

# Set the folder paths for input and output files
input_folder = r''
output_folder = r''
# Get a list of all Excel files in the input folder
excel_files = glob.glob(input_folder + '/*.xlsx')
# Process each Excel file
for file in excel_files:
    # Read the Excel file
    data = pd.read_excel(file)
    # Print the column names

    column_names = data.columns
    # Ask for the columns to delete
    columns_to_delete = ['AcctNo','Company','FieldID','First','Last','Address','City','State','Zip','Country','Grower','FarmID','LabID','Received','Processed','Matrix']


    # Delete the specified columns
    data.drop(columns=columns_to_delete, inplace=True)
    #Print the column names
    column_names = data.columns
    for name in column_names:

    # Ask for the sampleid column
     sampleid_column = 'SampleID'

    # Extract the data based on the fieldid column
    if sampleid_column in column_names:
        # Get the first three characters of the fieldid column as the farm ID
        farm_id = data[sampleid_column].apply(lambda x:x.split('-')[0])

        # Create new columns based on the farm ID and perform the required transformations
        block_key={
                'A':'1',
                'B':'2',
                'C':'3',
                'D':'4'

            }
        crop_key = {
            'T1': 'TO',
            'T2': 'TO',
            'T3': 'TO',
            'P1': 'PT',
            'P2': 'PT',
            'B1': 'GB',
            'B2': 'GB'
        }

        site_key = {
            'T1': 'LL', #
            'T2': 'PI', #
            'T3': 'LE', #
            'P1': 'TI', #
            'P2': 'TL', #
            'B1': 'CC', #
            'B2': 'HW'  #
        }
        season_key = {
            'T1': 'F3', # 09/29-30/2023
            'T2': 'W3', # 11/07/2023
            'T3': 'W3', #
            'P1': 'W3', # 11/02/2023
            'P2': 'W3', #
            'B1': 'W3', # 11/22/2023
            'B2': 'W3' #
        }
        spatial_key = {
                'DR': 'DR',  # Ditch row
                'FN': 'FN',  # First neighbor
                'SN': 'SN' ,  # Second neighbor
                'ND': 'ND'   # Second neighbor
            }
        farm_ids = set(farm_id.tolist())
        if 'T1' in farm_ids or  'T2'in farm_ids or 'T3'in farm_ids:

            depth_key = {
                'S1': '15', #6 inch
                'S2': '30', #12 inch
                'S3': '45' #18 inch
            }
            treatment_key = {
                '1': '0',
                '2': '50',
                '3': '75',
                '4': '100',
                '5': '150',
                '6': '200'

            }

        elif 'P1' in farm_ids or 'P2' in farm_ids:

            depth_key = {
                'S1': '20', #8 inch
                'S2': '40', #16 inch
                'S3': '60'  #24 inch
            }
            treatment_key = {
                '1': '0',
                '2': '46',
                '3': '92',
                '4': '137',
                '5': '183',
                '6': '229'

            }
        else:

            depth_key = {
                'S1': '15', #6 inch
                'S2': '30', #12 inch
                'S3': '45'  #18 inch
            }
            treatment_key = {
                '1': '0',
                '2': '40',
                '3': '80',
                '4': '120',
                '5': '113L'
            }
        # Create new columns based on the farm ID and perform the required transformations
        data["Crop"] = farm_id.apply(lambda x: crop_key.get(x, None))
        data["site"] = farm_id.apply(lambda x: site_key.get(x, None))
        data["season"] = farm_id.apply(lambda x: season_key.get(x, None))

         # Convert "Sampling Date" to a datetime object
        data["Sampling Date"] = pd.to_datetime(data[sampleid_column].apply(lambda x: x.split('-')[5]), format='%m%d%Y').dt.date

        data["sample_type"] = "S"
        # Extract the sample ID and create separate columns
        data["Block"] = data[sampleid_column].apply(lambda x:  block_key.get(x.split('-')[1] if isinstance(x, str) else None))
        data["Treatment"] = data[sampleid_column].apply(lambda x: treatment_key.get(x.split('-')[2] if isinstance(x, str) else None))
        data["Row_type"] = data[sampleid_column].apply(lambda x: spatial_key.get(x.split('-')[3], None))
        data["Depth"] = data[sampleid_column].apply(lambda x: depth_key.get(x.split('-')[4], None))
        data["Time"] = data.apply(lambda row: row.get("Time", "N/A"), axis=1)
        # Create the new_sample_ID column
        data["new_sample_ID"] = data.apply(lambda row: f"{row['Crop']}-{row['site']}-{row['season']}-S{row.name + 1}", axis=1)

        # Sort the columns
        column_order = [
            sampleid_column,
            "new_sample_ID",
            "Sampling Date",
            "Crop",
            "site",
            "season",
            "Treatment",
            "Block",
            "Row_type",
            "Depth",
            "Time",
            "sample_type"
        ]
        # Create a list of columns that are not in column_order
        remaining_columns = [col for col in column_names if col not in column_order]

        # Append the remaining columns to the column_order list
        column_order += remaining_columns

        # Reorder the columns in the DataFrame based on column_order
        data = data[column_order]


        # Print the modified data
        print("\nModified data:")
        print(data)
       # Create the output file path
        file_name = os.path.basename(file)
        output_file = os.path.join(output_folder, file_name.replace('.xlsx', '_Masterfile_preprocessed.xlsx'))
        # Save the modified data to a new Excel file in the outp
        data.to_excel(output_file, index=False)
        print(f"Processed data saved to '{output_file}'.")
    else:
      print("Fieldid column not found in the Excel file.")